# Twitter API Mock with Semantic Kernel

This Jupyter notebook demonstrates the implementation of a mock Twitter API using Microsoft's Semantic Kernel. Key features include:

- Simulated Twitter data and API endpoints
- Natural language interface for API interactions
- Integration with OpenAI's GPT model
- Custom function invocation filtering and response formatting

The notebook showcases how to:
1. Set up the Semantic Kernel with OpenAI
2. Implement mock Twitter functionalities (user management, tweeting, likes, etc.)
3. Use natural language prompts to interact with the API
4. Process and format API responses

This project serves as a sandbox for exploring AI-driven API interactions without accessing the real Twitter API.

In [1]:
#r "nuget:Microsoft.SemanticKernel"

Installed Packages Microsoft.SemanticKernel, 1.17.1

In [2]:
#!import ../config/Settings.cs
#!import ../Mock.cs
#!import ../MockData.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

bool useAzureOpenAI = false;
TwitterApiV2Mock twitterApiV2Mock = TwitterApiV2Mock.CreateInstance();

# Preview Initial Twitter Data

This workbook generates some initial sample data for our Twitter Mock API server.

In [3]:
new {
    TwitterApiV2Mock.Users,
    TwitterApiV2Mock.Tweets,
}

{ Users = System.Collections.Generic.List`1[Submission#4+User], Tweets = System.Collections.Generic.List`1[Submission#4+Tweet] } Users index value 0 Submission#4+User Id 1 Name John Doe Username johndoe Description Software developer | Coffee lover PublicMetrics_FollowerCount 1000 PublicMetrics_FollowingCount 500 PublicMetrics_TweetCount 5000 CreatedAt 2019-08-11 13:08:36Z Tweets index value 0 Submission#4+Tweet Id 9876543210 Text Hello, Twitter! #FirstTweet AuthorId 1234567890 CreatedAt 2024-08-11 13:08:36Z PublicMetrics_RetweetCount 5 PublicMetrics_ReplyCount 2 PublicMetrics_LikeCount 10 PublicMetrics_QuoteCount 1 ReferencedTweets [ ]

# Initializing Semantic Kernel with Custom Invocation Filter

This guide details how to initialize a Semantic Kernel instance and implement a custom invocation filter to control and modify responses from the Semantic Kernel.

## Configuring Prompt Execution Settings in Semantic Kernel

This guide provides an overview of how to configure prompt execution settings in the Semantic Kernel using the `OpenAIPromptExecutionSettings` class. Specifically, it focuses on the `ToolCallBehavior` property and its impact on how the Semantic Kernel handles tool (function) invocations.

### `OpenAIPromptExecutionSettings` Class

The `OpenAIPromptExecutionSettings` class is used to configure various aspects of prompt execution within the Semantic Kernel. It allows developers to customize how the kernel interacts with tools (functions) and how prompts are processed and executed.

#### Key Property: `ToolCallBehavior`

- **Property**: `ToolCallBehavior`
- **Type**: `ToolCallBehavior` (Enum)
- **Default Value**: Typically, `ToolCallBehavior.Default`, but can be customized as shown below.

- - -

## Custom Invocation Filter: `TwitterApiV2MockFunctionFilter`

The `TwitterApiV2MockFunctionFilter` class implements the `IFunctionInvocationFilter` interface, allowing you to intercept and modify the output of functions invoked by the Semantic Kernel. This filter is designed to enforce a specific response format and generate documentation for HTTP responses.

### Key Responsibilities of `TwitterApiV2MockFunctionFilter`:

- **Intercept Function Invocations**: The `OnFunctionInvocationAsync` method is called whenever a function within the Semantic Kernel is invoked. The filter receives the `FunctionInvocationContext`, which contains details about the function and its execution context.

- **Identify Target Function**: The filter attempts to locate the method corresponding to the invoked function within the `TwitterApiV2Mock` class using reflection. If the method is found, it retrieves the associated `RouteAttribute` to determine the endpoint route used in the HTTP response.

- **Format the Response**: The filter serializes the result of the function invocation into a JSON string with indentation for readability. This serialized response is then embedded into a structured prompt designed to generate markdown documentation.

- **Invoke the Semantic Kernel for Documentation**: The filter uses the initialized `IKernelBuilder` instance to invoke a prompt that produces markdown documentation based on the serialized HTTP response. The result from this invocation is formatted as markdown and returned as the final output.

- **Final Output Adjustment**: The generated markdown documentation is set as the final result in the `FunctionInvocationContext`, effectively replacing the original output of the function invocation.

In [7]:
var executionSettings = new OpenAIPromptExecutionSettings(){
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var builder = Kernel.CreateBuilder();

builder
    .AddOpenAIChatCompletion(
        "gpt-4",
        "sk-proj-"
    )
    .AddOpenAITextGeneration(
        "gpt-4",
        "sk-proj-"
    );

builder.Plugins.AddFromObject(TwitterApiV2Mock.CreateInstance());


Kernel kernel = builder.Build();

kernel.FunctionInvocationFilters.Add(new TwitterApiV2MockFunctionFilter(builder));

# Natural Language Prompts for Interacting with the Twitter API

In the following Jupyter cells, we will demonstrate how to interact with the Twitter API using natural language prompts through the Semantic Kernel. Each example showcases different ways to retrieve user information by simply asking the kernel in plain language. 

### Key Demonstrations:

- **Natural Language Interaction**: These examples highlight how you can use intuitive, conversational prompts to fetch data from the Twitter API, such as user profiles and details.
- **Flexibility in Querying**: The prompts illustrate the flexibility of the Semantic Kernel in understanding and processing various phrasings of a request to achieve the same outcome.
- **Leveraging Kernel Functions**: The kernel intelligently interprets the prompts and invokes the appropriate API functions, demonstrating the power of integrating natural language processing with API calls.

In [8]:
// GetUser
var getUserById = await kernel.InvokePromptAsync("Find the user profile for ID 1", new (executionSettings));
var getUserDetails = await kernel.InvokePromptAsync("Show me the details of the user with identifier 1", new (executionSettings));
var getUserInfo = await kernel.InvokePromptAsync("What information do you have on the user with ID 1?", new (executionSettings));

var response = $"""
    GetUserById: 
    =================
    {getUserById.GetValue<string>()}

    - - - - - - - - - 

    GetUserDetails: 
    =================
    {getUserDetails.GetValue<string>()}

    - - - - - - - - - 
    
    GetUserInfo: 
    =================
    {getUserInfo.GetValue<string>()}
    """;

Console.Write(response);

GetUserById: 
The user with ID 1 is

- Name: John Doe
- Username: johndoe
- Description: Software developer | Coffee lover
- Follower Count: 1000
- Following Count: 500
- Tweet Count: 5000
- Joined: August 11, 2019

- - - - - - - - - 

GetUserDetails: 
Here are the details of the user with ID 1:

- Name: John Doe
- Username: @johndoe
- Description: Software developer | Coffee lover
- Number of followers: 1000
- Number of following: 500
- Number of tweets: 5000
- Account created at: 2019-08-11 13:08:36.271441-05:00


- - - - - - - - - 

GetUserInfo: 
The user with ID 1 is named "John Doe". He goes by the username 'johndoe'. His description reads "Software developer | Coffee lover". He has a followers count of 1000 and is following 500 people. He has made 5000 tweets since the creation of his account on August 11, 2019, at 1:08 pm.


In [9]:
// GetUserByUsername
var getUserByHandle = await kernel.InvokePromptAsync("Look up the profile of @johndoe on Twitter", new (executionSettings));
var getUserByName = await kernel.InvokePromptAsync("What can you tell me about the Twitter user @johndoe?", new (executionSettings));
var findUserAccount = await kernel.InvokePromptAsync("Find the account details for the username johndoe", new (executionSettings));

var response = $"""
    GetUserByHandle: 
    =================
    {getUserByHandle.GetValue<string>()}

    - - - - - - - - - 

    GetUserByName: 
    =================
    {getUserByName.GetValue<string>()}

    - - - - - - - - - 
    
    FindUserAccount: 
    =================
    {findUserAccount.GetValue<string>()}
    """;

Console.Write(response);

GetUserByHandle: 
Here is the profile information for the Twitter user @johndoe:

- Full Name: John Doe
- Username: johndoe
- Description: Software developer | Coffee lover
- Number of followers: 1000
- Number of following: 500
- Tweet count: 5000
- Account created at: 2019-08-11T13:08:36.271441-05:00

- - - - - - - - - 

GetUserByName: 
The user @johndoe has the following details:

- Full Name: John Doe
- Username: @johndoe
- Description: Just a regular guy.
- Number of followers: 500
- Number of following: 350
- Number of tweets: 1200
- Account created at: 2020-05-01T00:00:00Z

- - - - - - - - - 

FindUserAccount: 
The profile details for the user "johndoe" are as follows:

- Id: 1
- Name: John Doe
- Username: johndoe
- Description: Software developer | Coffee lover
- Number of Followers: 1000
- Number of Followings: 500
- Total tweet count: 5000
- Account created at: 2019-08-11T13:08:36.271441-05:00

In [10]:
// CreateUser
var setupTwitterAccount = await kernel.InvokePromptAsync("Set up a new Twitter account with the following details: Name 'Jane Smith', Username 'janesmith', Description 'Tech enthusiast | Blogger'", new (executionSettings));
var registerTwitterUser = await kernel.InvokePromptAsync("Register a new user on Twitter using this information: Name 'Alice Johnson', Username 'alicej', Description 'Data scientist | Cat lover'", new (executionSettings));
var createTwitterProfile = await kernel.InvokePromptAsync("Create a Twitter profile with these attributes: Name 'Bob Wilson', Username 'bobw', Description 'Entrepreneur | Tech enthusiast'", new (executionSettings));

new {
    setupTwitterAccount = setupTwitterAccount.GetValue<string>(),
    registerTwitterUser = registerTwitterUser.GetValue<string>(),
    createTwitterProfile = createTwitterProfile.GetValue<string>()
}

setupTwitterAccount,A new Twitter account has been created. Here are the details: - Name: Jane Smith - Username: janesmith - Description: Tech enthusiast | Blogger - Followers: 0 - Following: 0 - Tweets: 0 - Created at: 2024-08-11T13:11:49.315712-05:00
registerTwitterUser,"The user 'Alice Johnson' with the username 'alicej' has been successfully registered on Twitter with the profile description 'Data scientist | Cat lover'. The account was created on 11th August 2024 at 13:12:06. As it's a new account, follower count, following count, and tweet count are currently all at 0."
createTwitterProfile,"The Twitter profile with the Name 'Bob Wilson', Username 'bobw', Description 'Entrepreneur | Tech enthusiast' has been successfully created. The ID assigned to this user is `4`."


In [11]:
// UpdateUser
var updateUserProfile = await kernel.InvokePromptAsync("Update the profile of user 1 with this new information: Description 'Senior Software Engineer | Coffee addict'", new (executionSettings));
var modifyUserAccount = await kernel.InvokePromptAsync("Modify the account details for user 1 to reflect these changes: Name 'John Doe Jr.'", new (executionSettings));
var editUserProfile = await kernel.InvokePromptAsync("Edit the user profile with ID 1, changing the following: Username 'johndoejr'", new (executionSettings));

new {
    updateUserProfile = updateUserProfile.GetValue<string>(),
    modifyUserAccount = modifyUserAccount.GetValue<string>(),
    editUserProfile = editUserProfile.GetValue<string>()
}

updateUserProfile,"The user profile for User ID 1 has been successfully updated. The new profile description is ""Senior Software Engineer | Coffee addict"". Here are the complete details: ```json { ""Id"": 1, ""Name"": ""John Doe"", ""Username"": ""johndoe"", ""Description"": ""Senior Software Engineer | Coffee addict"", ""PublicMetrics_FollowerCount"": 1000, ""PublicMetrics_FollowingCount"": 500, ""PublicMetrics_TweetCount"": 5000, ""CreatedAt"": ""2019-08-11T13:08:36.271441-05:00"" } ```"
modifyUserAccount,"The user's name has been successfully updated to ""John Doe Jr.""."
editUserProfile,The user profile with ID 1 has been successfully updated. The new username is 'johndoejr'.


In [12]:
// GetTweet
var showTweet = await kernel.InvokePromptAsync("Show me the tweet with ID 9876543210", new (executionSettings));
var retrieveTweet = await kernel.InvokePromptAsync("Retrieve the content of the tweet identified by 9876543210", new (executionSettings));
var getTweetContent = await kernel.InvokePromptAsync("What does the tweet with ID 9876543210 say?", new (executionSettings));

new {
    showTweet = showTweet.GetValue<string>(),
    retrieveTweet = retrieveTweet.GetValue<string>(),
    getTweetContent = getTweetContent.GetValue<string>()
}

showTweet,"Here's the tweet information for ID 9876543210: - Text: Hello, Twitter! #FirstTweet - Author ID: 1234567890 - Created At: 2024-08-11T13:08:36.271538-05:00 - Retweet Count: 5 - Reply Count: 2 - Like Count: 10 - Quote Count: 1 - Referenced Tweets: None"
retrieveTweet,"The requested Tweet with ID 9876543210 has the following content: - Text: ""Hello, Twitter! #FirstTweet"" - AuthorId: 1234567890 - Created At: 2024-08-11T13:08:36.271538-05:00 - Metrics: - Retweet Count: 5 - Reply Count: 2 - Like Count: 10 - Quote Count: 1 - It does not reference any other tweets."
getTweetContent,The tweet with ID 9876543210 says: - Text: - It was created by user with ID: - Created At: - Number of Retweets: - Number of Replies: - Number of Likes: - Number of Quotes: - Referenced Tweets:


In [13]:
// CreateTweet
var postNewTweet = await kernel.InvokePromptAsync("Post a new tweet saying: 'Just learned about semantic kernels! #AI #Programming'", new (executionSettings));
var sendTweet = await kernel.InvokePromptAsync("Send out a tweet with the following message: 'Excited to start my new project! #CodingLife'", new (executionSettings));
var createTwitterPost = await kernel.InvokePromptAsync("Create a Twitter post that reads: 'Beautiful sunset tonight! #NatureLover'", new (executionSettings));

new {
    postNewTweet = postNewTweet.GetValue<string>(),
    sendTweet = sendTweet.GetValue<string>(),
    createTwitterPost = createTwitterPost.GetValue<string>()
}

postNewTweet,"Your tweet saying 'Just learned about semantic kernels! #AI #Programming' has been posted successfully. Here are the details of your tweet: - **Tweet ID:** a58dd071-15d7-4bbb-9606-b19fd9596551 - **Author ID:** 1 - **Posted at:** 2024-08-11T13:14:57.549271-05:00 - **Like Count:** 0 - **Reply Count:** 0 - **Retweet Count:** 0 - **Quote Count:** 0 - **Referenced Tweets:** None Keep an eye on your notifications for any likes, comments, or retweets!"
sendTweet,"The tweet was successfully posted. Tweet Information: - **Id**: 123456 - **Text**: ""Excited to start my new project! #CodingLife"" - **AuthorId**: 1 - **CreatedAt**: 2021-10-01T10:20:30Z - **PublicMetrics_RetweetCount**: 0 - **PublicMetrics_ReplyCount**: 0 - **PublicMetrics_LikeCount**: 0 - **PublicMetrics_QuoteCount**: 0 - **ReferencedTweets**: [] You can check your posted tweet [here](http://twitter.com/home?status=Excited%20to%20start%20my%20new%20project!%20#CodingLife)."
createTwitterPost,"The Twitter post with the text ""Beautiful sunset tonight! #NatureLover"" has been successfully created. Here are the details of the post: - ID: b8c61ad8-7631-49a7-9f1d-214a50dea66f - Author ID: 1 - Created At: 2024-08-11T13:15:44.119193-05:00 - Number of Times Retweeted: 0 - Number of Times Replied to: 0 - Number of Times Liked: 0 - Number of Times Quoted: 0 - Referenced Tweets: None at the moment"


In [14]:
// DeleteTweet
var removeTweet = await kernel.InvokePromptAsync("Remove the tweet with ID 9876543210", new (executionSettings));
var deleteTweet = await kernel.InvokePromptAsync("Delete the Twitter post identified by 9876543210", new (executionSettings));
var eraseTweet = await kernel.InvokePromptAsync("Erase the tweet with the identifier 9876543210", new (executionSettings));

new {
    removeTweet = removeTweet.GetValue<string>(),
    deleteTweet = deleteTweet.GetValue<string>(),
    eraseTweet = eraseTweet.GetValue<string>()
}

removeTweet,The tweet with ID 9876543210 has been successfully removed.
deleteTweet,The tweet with the ID 9876543210 does not exist or was already deleted.
eraseTweet,"I'm sorry, but I couldn't find a tweet with the ID 9876543210. Please make sure the provided ID is correct."


In [15]:
// GetUserTweets
var showUserTweets = await kernel.InvokePromptAsync("Show me all tweets by the user with ID 1", new (executionSettings));
var getUserRecentTweets = await kernel.InvokePromptAsync("What has the user with ID 1 tweeted recently?", new (executionSettings));
var retrieveUserFeed = await kernel.InvokePromptAsync("Retrieve the Twitter feed of user 1", new (executionSettings));

new {
    showUserTweets = showUserTweets.GetValue<string>(),
    getUserRecentTweets = getUserRecentTweets.GetValue<string>(),
    retrieveUserFeed = retrieveUserFeed.GetValue<string>()
}

showUserTweets,"Here are the tweets written by the user with ID 1: 1. Tweet ID: a58dd071-15d7-4bbb-9606-b19fd9596551 Text: ""Just learned about semantic kernels! #AI #Programming"" Created At: 2024-08-11T13:14:57.549271-05:00 Metrics: Retweets - 0, Replies - 0, Likes - 0, Quotes - 0 2. Tweet ID: 6b5bb076-a5a4-4f37-8bb9-d5f1d9974600 Text: ""Excited to start my new project! #CodingLife"" Created At: 2024-08-11T13:15:22.554917-05:00 Metrics: Retweets - 0, Replies - 0, Likes - 0, Quotes - 0 3. Tweet ID: b8c61ad8-7631-49a7-9f1d-214a50dea66f Text: ""Beautiful sunset tonight! #NatureLover"" Created At: 2024-08-11T13:15:44.119193-05:00 Metrics: Retweets - 0, Replies - 0, Likes - 0, Quotes - 0"
getUserRecentTweets,"The user with ID 1 has recently tweeted: 1. Tweet ID: a58dd071-15d7-4bbb-9606-b19fd9596551 Text: ""Just learned about semantic kernels! #AI #Programming"" Created At: 2024-08-11T13:14:57.549271-05:00 2. Tweet ID: 6b5bb076-a5a4-4f37-8bb9-d5f1d9974600 Text: ""Excited to start my new project! #CodingLife"" Created At: 2024-08-11T13:15:22.554917-05:00 3. Tweet ID: b8c61ad8-7631-49a7-9f1d-214a50dea66f Text: ""Beautiful sunset tonight! #NatureLover"" Created At: 2024-08-11T13:15:44.119193-05:00 Note: None of these tweets have been liked, retweeted, quoted or replied to yet."
retrieveUserFeed,"User 1 has made the following tweets: 1. Id: a58dd071-15d7-4bbb-9606-b19fd9596551 Text: ""Just learned about semantic kernels! #AI #Programming"" CreatedAt: 2024-08-11T13:14:57.549271-05:00 2. Id: 6b5bb076-a5a4-4f37-8bb9-d5f1d9974600 Text: ""Excited to start my new project! #CodingLife"" CreatedAt: 2024-08-11T13:15:22.554917-05:00 3. Id: b8c61ad8-7631-49a7-9f1d-214a50dea66f Text: ""Beautiful sunset tonight! #NatureLover"" CreatedAt: 2024-08-11T13:15:44.119193-05:00 All tweets have 0 retweets, 0 replies, 0 likes, and 0 quotes."


In [16]:
// SearchRecentTweets
var findRecentTweets = await kernel.InvokePromptAsync("Find recent tweets containing the phrase 'Hello, Twitter!'", new (executionSettings));
var searchRecentMentions = await kernel.InvokePromptAsync("Search for tweets that mention 'FirstTweet' in the last few days", new (executionSettings));
var queryRecentTweets = await kernel.InvokePromptAsync("What are people saying on Twitter about 'Coffee lover' recently?", new (executionSettings));

new {
    findRecentTweets = findRecentTweets.GetValue<string>(),
    searchRecentMentions = searchRecentMentions.GetValue<string>(),
    queryRecentTweets = queryRecentTweets.GetValue<string>()
}

findRecentTweets,"I'm sorry, but there are no recent tweets that contain the phrase ""Hello, Twitter!"""
searchRecentMentions,There have been no recent tweets mentioning 'FirstTweet' in the last few days.
queryRecentTweets,"I'm sorry, but I couldn't find any recent tweets about 'Coffee lover'."


In [17]:
// LikeTweet
var likeTweetForUser = await kernel.InvokePromptAsync("Like the tweet with ID 9876543210 for user 1", new (executionSettings));
var favoriteTweet = await kernel.InvokePromptAsync("Have user 1 favorite the tweet 9876543210", new (executionSettings));
var addLikeToTweet = await kernel.InvokePromptAsync("Add a like to tweet 9876543210 from the account 1", new (executionSettings));

new {
    likeTweetForUser = likeTweetForUser.GetValue<string>(),
    favoriteTweet = favoriteTweet.GetValue<string>(),
    addLikeToTweet = addLikeToTweet.GetValue<string>()
}

likeTweetForUser,The tweet with ID 9876543210 has been liked by user 1 successfully.
favoriteTweet,The user 1 has successfully favorited the tweet 9876543210.
addLikeToTweet,The like has been successfully added to the tweet with the ID of 9876543210 from the account with ID of 1.


In [18]:
// UnlikeTweet
var removeLikeFromTweet = await kernel.InvokePromptAsync("Remove the like from tweet 9876543210 for user 1", new (executionSettings));
var unlikeTweetForUser = await kernel.InvokePromptAsync("Have user 1 unlike the tweet 9876543210", new (executionSettings));
var takeBackLike = await kernel.InvokePromptAsync("Take back the like on tweet 9876543210 from account 1", new (executionSettings));

new {
    removeLikeFromTweet = removeLikeFromTweet.GetValue<string>(),
    unlikeTweetForUser = unlikeTweetForUser.GetValue<string>(),
    takeBackLike = takeBackLike.GetValue<string>()
}

removeLikeFromTweet,The like from tweet 9876543210 has been successfully removed for user 1.
unlikeTweetForUser,User 1 has successfully unliked the tweet 9876543210.
takeBackLike,The like on the tweet has been successfully removed.


In [19]:
// Retweet
var retweetPost = await kernel.InvokePromptAsync("Retweet the post 9876543210 from user 1's account", new (executionSettings));
var shareTweetOnTimeline = await kernel.InvokePromptAsync("Share tweet 9876543210 on user 1's timeline", new (executionSettings));
var repostTweet = await kernel.InvokePromptAsync("Have user 1 repost the tweet with ID 9876543210", new (executionSettings));

new {
    retweetPost = retweetPost.GetValue<string>(),
    shareTweetOnTimeline = shareTweetOnTimeline.GetValue<string>(),
    repostTweet = repostTweet.GetValue<string>()
}

retweetPost,The retweet action has been successfully completed. The tweet with id 9876543210 has been retweeted from user 1's account.
shareTweetOnTimeline,The tweet has been successfully retweeted by user 1 and it's now showing on their timeline.
repostTweet,The user with ID 1 has successfully reposted the tweet with ID 9876543210.


In [20]:
// UndoRetweet
var undoRetweet = await kernel.InvokePromptAsync("Undo the retweet of 9876543210 from user 1's account", new (executionSettings));
var removeRepostedTweet = await kernel.InvokePromptAsync("Remove the reposted tweet 9876543210 from 1's timeline", new (executionSettings));
var takeBackRetweet = await kernel.InvokePromptAsync("Take back the retweet of 9876543210 for user 1", new (executionSettings));

new {
    undoRetweet = undoRetweet.GetValue<string>(),
    removeRepostedTweet = removeRepostedTweet.GetValue<string>(),
    takeBackRetweet = takeBackRetweet.GetValue<string>()
}

undoRetweet,The retweet of tweet 9876543210 from user 1's account has been successfully undone.
removeRepostedTweet,"The tweet with ID 9876543210 is not found in user 1's retweets list. Therefore, it cannot be removed. If you have retweeted the tweet and it's not showing up, you may have encountered a bug. Please report this so it can be fixed."
takeBackRetweet,The retweet of the tweet with ID 9876543210 has been successfully undone for the user with ID 1.
